In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import io
from torchvision.models import resnet18, ResNet18_Weights, resnet50, resnet101, ResNet101_Weights, ResNet50_Weights
from tqdm import tqdm  # For progress bars
import time

In [1]:
import os
import random
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet18
import cv2  # For resizing the Grad-CAM heatmap

# Paths
model_path = "/Users/paolocadei/Desktop/GeolocationGuesserAI/model.pth"
dataset_path = "/Users/paolocadei/Desktop/GeolocationGuesserAI/Streetview_Image_Dataset"
output_dir = "/Users/paolocadei/Desktop/GeolocationGuesserAI/grad-cam"

# Load pre-trained ResNet model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18(pretrained=False)
num_features = model.fc.in_features

# Temporarily set the fc layer to match the checkpoint dimensions
model.fc = torch.nn.Linear(num_features, 16)  # Match the checkpoint's number of output classes
model.load_state_dict(torch.load(model_path, map_location=device))

# Replace the fc layer for the current task (4 classes)
model.fc = torch.nn.Linear(num_features, 4)
model.to(device)
model.eval()

# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Randomly select 100 images from the dataset
image_files = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]
random_images = random.sample(image_files, 100)  # Change to 100 images

# Grad-CAM setup
target_layers = [model.layer4[-1]]  # Last convolutional layer
os.makedirs(output_dir, exist_ok=True)

# Process each image and generate Grad-CAM visualizations
for idx, image_file in enumerate(random_images):
    image_path = os.path.join(dataset_path, image_file)
    image = Image.open(image_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)  # Prepare input tensor

    # Specify target class (optional; defaults to predicted class if None)
    targets = None  # Alternatively, specify class index with ClassifierOutputTarget(class_index)

    # Construct Grad-CAM object and generate heatmap
    with GradCAM(model=model, target_layers=target_layers) as cam:
        grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
        grayscale_cam = grayscale_cam[0, :]  # Extract heatmap for the first image in the batch

        # Resize Grad-CAM heatmap to match the original image dimensions
        grayscale_cam_resized = cv2.resize(grayscale_cam, (image.width, image.height))

        # Overlay heatmap on the original image
        image_np = np.array(image) / 255.0  # Normalize image to range [0, 1]
        visualization = show_cam_on_image(image_np, grayscale_cam_resized, use_rgb=True)

        # Save the Grad-CAM visualization
        output_path = os.path.join(output_dir, f"gradcam_{idx}.jpg")
        Image.fromarray(visualization).save(output_path)
        print(f"Saved Grad-CAM for {image_file} to {output_path}")

print(f"Grad-CAM visualizations for 100 images saved in {output_dir}.")


/Users/paolocadei/opt/anaconda3/envs/TF/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/paolocadei/opt/anaconda3/envs/TF/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Saved Grad-CAM for 2939.png to /Users/paolocadei/Desktop/GeolocationGuesserAI/grad-cam/gradcam_0.jpg
Saved Grad-CAM for 14107.png to /Users/paolocadei/Desktop/GeolocationGuesserAI/grad-cam/gradcam_1.jpg
Saved Grad-CAM for 11758.png to /Users/paolocadei/Desktop/GeolocationGuesserAI/grad-cam/gradcam_2.jpg
Saved Grad-CAM for 11098.png to /Users/paolocadei/Desktop/GeolocationGuesserAI/grad-cam/gradcam_3.jpg
Saved Grad-CAM for 20388.png to /Users/paolocadei/Desktop/GeolocationGuesserAI/grad-cam/gradcam_4.jpg
Saved Grad-CAM for 10644.png to /Users/paolocadei/Desktop/GeolocationGuesserAI/grad-cam/gradcam_5.jpg
Saved Grad-CAM for 1047.png to /Users/paolocadei/Desktop/GeolocationGuesserAI/grad-cam/gradcam_6.jpg
Saved Grad-CAM for 9265.png to /Users/paolocadei/Desktop/GeolocationGuesserAI/grad-cam/gradcam_7.jpg
Saved Grad-CAM for 11571.png to /Users/paolocadei/Desktop/GeolocationGuesserAI/grad-cam/gradcam_8.jpg
Saved Grad-CAM for 13334.png to /Users/paolocadei/Desktop/GeolocationGuesserAI/grad-c

In [ ]:
import os
import pandas as pd
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet18
import cv2  # For resizing the Grad-CAM heatmap

# Paths
model_path = "/Users/paolocadei/Desktop/GeolocationGuesserAI/model.pth"
dataset_path = "/Users/paolocadei/Desktop/GeolocationGuesserAI/Streetview_Image_Dataset"
output_dir = "/Users/paolocadei/Desktop/GeolocationGuesserAI/grad-cam"
csv_path = "/Users/paolocadei/Desktop/GeolocationGuesserAI/models/Resnet18_Frozen/test_predictions.csv"  # Path to the CSV file

# Load mismatched predictions
data = pd.read_csv(csv_path)
mismatched_images = data[data["model_prediction"] != data["sub-region"]]["image_name"].tolist()
print(f"Found {len(mismatched_images)} mismatched images.")

# Load pre-trained ResNet model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18(pretrained=False)
num_features = model.fc.in_features

# Temporarily set the fc layer to match the checkpoint dimensions
model.fc = torch.nn.Linear(num_features, 16)  # Match the checkpoint's number of output classes
model.load_state_dict(torch.load(model_path, map_location=device))

# Replace the fc layer for the current task (4 classes)
model.fc = torch.nn.Linear(num_features, 4)
model.to(device)
model.eval()

# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Grad-CAM setup
target_layers = [model.layer4[-1]]  # Last convolutional layer
os.makedirs(output_dir, exist_ok=True)

# Process each mismatched image and generate Grad-CAM visualizations
for idx, image_file in enumerate(mismatched_images):
    image_path = os.path.join(dataset_path, image_file)
    if not os.path.exists(image_path):
        print(f"Image {image_file} not found, skipping...")
        continue

    image = Image.open(image_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)  # Prepare input tensor

    # Specify target class (optional; defaults to predicted class if None)
    targets = None  # Alternatively, specify class index with ClassifierOutputTarget(class_index)

    # Construct Grad-CAM object and generate heatmap
    with GradCAM(model=model, target_layers=target_layers) as cam:
        grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
        grayscale_cam = grayscale_cam[0, :]  # Extract heatmap for the first image in the batch

        # Resize Grad-CAM heatmap to match the original image dimensions
        grayscale_cam_resized = cv2.resize(grayscale_cam, (image.width, image.height))

        # Overlay heatmap on the original image
        image_np = np.array(image) / 255.0  # Normalize image to range [0, 1]
        visualization = show_cam_on_image(image_np, grayscale_cam_resized, use_rgb=True)

        # Save the Grad-CAM visualization
        output_path = os.path.join(output_dir, f"gradcam_mismatch_{idx}.jpg")
        Image.fromarray(visualization).save(output_path)
        print(f"Saved Grad-CAM for mismatched image {image_file} to {output_path}")

print(f"Grad-CAM visualizations for mismatched images saved in {output_dir}.")


In [ ]:
import os
import pandas as pd
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet18
import cv2  # For resizing the Grad-CAM heatmap

# Paths
model_path = "/Users/paolocadei/Desktop/GeolocationGuesserAI/model.pth"
dataset_path = "/Users/paolocadei/Desktop/GeolocationGuesserAI/Streetview_Image_Dataset"
output_dir = "/Users/paolocadei/Desktop/GeolocationGuesserAI/grad-cam-correct"
csv_path = "/Users/paolocadei/Desktop/GeolocationGuesserAI/models/Resnet18_Frozen/train_predictions.csv"  # Path to the CSV file

# Load correctly classified predictions
data = pd.read_csv(csv_path)
correctly_classified_images = data[data["model_prediction"] == data["sub-region"]]["image_name"].tolist()[:100]
print(f"Found {len(correctly_classified_images)} correctly classified images (first 100 selected).")

# Load pre-trained ResNet model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18(pretrained=False)
num_features = model.fc.in_features

# Temporarily set the fc layer to match the checkpoint dimensions
model.fc = torch.nn.Linear(num_features, 16)  # Match the checkpoint's number of output classes
model.load_state_dict(torch.load(model_path, map_location=device))

# Replace the fc layer for the current task (4 classes)
model.fc = torch.nn.Linear(num_features, 4)
model.to(device)
model.eval()

# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Grad-CAM setup
target_layers = [model.layer4[-1]]  # Last convolutional layer
os.makedirs(output_dir, exist_ok=True)

# Process each correctly classified image and generate Grad-CAM visualizations
for idx, image_file in enumerate(correctly_classified_images):
    image_path = os.path.join(dataset_path, image_file)
    if not os.path.exists(image_path):
        print(f"Image {image_file} not found, skipping...")
        continue

    image = Image.open(image_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)  # Prepare input tensor

    # Specify target class (optional; defaults to predicted class if None)
    targets = None  # Alternatively, specify class index with ClassifierOutputTarget(class_index)

    # Construct Grad-CAM object and generate heatmap
    with GradCAM(model=model, target_layers=target_layers) as cam:
        grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
        grayscale_cam = grayscale_cam[0, :]  # Extract heatmap for the first image in the batch

        # Resize Grad-CAM heatmap to match the original image dimensions
        grayscale_cam_resized = cv2.resize(grayscale_cam, (image.width, image.height))

        # Overlay heatmap on the original image
        image_np = np.array(image) / 255.0  # Normalize image to range [0, 1]
        visualization = show_cam_on_image(image_np, grayscale_cam_resized, use_rgb=True)

        # Save the Grad-CAM visualization
        output_path = os.path.join(output_dir, f"gradcam_correct_{idx}.jpg")
        Image.fromarray(visualization).save(output_path)
        print(f"Saved Grad-CAM for correctly classified image {image_file} to {output_path}")

print(f"Grad-CAM visualizations for correctly classified images saved in {output_dir}.")
